# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np
from abstract_matrix import AbstractMatrix

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4382390288
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

## Test Matrices

In [3]:
arr0 = np.array([['{1, 2}', '{0, 1, 2}'],
                 ['{0, 2}', '{}']], dtype='<U32')

mat0 = AbstractMatrix(arr0, ps3)
mat0

PSRing3 Matrix:
[['{1, 2}' '{0, 1, 2}']
 ['{0, 2}' '{}']]

In [4]:
arr1 = np.array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
                 ['{0, 2}', '{}', '{1}'],
                 ['{0}', '{1}', '{0, 1}']], dtype='<U32')

mat1 = AbstractMatrix(arr1, ps3)
mat1

PSRing3 Matrix:
[['{1, 2}' '{0, 1, 2}' '{0, 2}']
 ['{0, 2}' '{}' '{1}']
 ['{0}' '{1}' '{0, 1}']]

In [5]:
arr2 = [['{0, 1}', '{0, 2}', '{1, 2}'],
        ['{0}'   , '{1, 2}',    '{2}'],
        ['{0, 1}',     '{}',     '{}']]

mat2 = AbstractMatrix(arr2, ps3)
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

In [6]:
arr3 = np.array([['{}', '{0, 1, 2}', '{0, 1, 2}', '{}'],
                 ['{0, 1}', '{0, 2}', '{1, 2}', '{2}'],
                 ['{0, 2}', '{}', '{}', '{0, 1}'],
                 ['{1}', '{0}', '{0, 2}', '{}']], dtype='<U32')

mat3 = AbstractMatrix(arr3, ps3)
mat3

PSRing3 Matrix:
[['{}' '{0, 1, 2}' '{0, 1, 2}' '{}']
 ['{0, 1}' '{0, 2}' '{1, 2}' '{2}']
 ['{0, 2}' '{}' '{}' '{0, 1}']
 ['{1}' '{0}' '{0, 2}' '{}']]

In [7]:
arr4 = np.array([['{1, 2}']], dtype='<U32')

mat4 = AbstractMatrix(arr4, ps3)
mat4

PSRing3 Matrix:
[['{1, 2}']]

## Matrix Addition & Subtraction

In [8]:
mat1 + mat2

PSRing3 Matrix:
[['{0, 2}' '{1}' '{0, 1}']
 ['{2}' '{1, 2}' '{1, 2}']
 ['{1}' '{1}' '{0, 1}']]

In [9]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

The array shapes are not equal: (3, 3) != (4, 4)


In [10]:
mat2 - mat2

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']
 ['{}' '{}' '{}']]

In [11]:
try:
    mat2p3 = mat2 - mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

The array shapes are not equal: (3, 3) != (4, 4)


## Matrix Multiplication

In [12]:
mat1 * mat2

PSRing3 Matrix:
[['{1}' '{1}' '{1}']
 ['{0, 1}' '{0, 2}' '{2}']
 ['{1}' '{0, 1}' '{}']]

In [13]:
mat2 * mat1

PSRing3 Matrix:
[['{0, 1, 2}' '{0}' '{0, 1}']
 ['{2}' '{0}' '{0, 1}']
 ['{1}' '{0, 1}' '{0}']]

In [14]:
try:
    mat2x3 = mat2 * mat3
    print(mat2x3)
except Exception as exc:
    print(exc)

The array shapes are incompatible: 3 columns vs 4 rows


## Abstract Matrix of "Zeros"

In [15]:
matz = AbstractMatrix.zeros((2, 3), ps3)
matz

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']]

## Abstract Identity Matrix

In [37]:
AbstractMatrix.identity(4, ps3)

PSRing3 Matrix:
[['{0, 1, 2}' '{}' '{}' '{}']
 ['{}' '{0, 1, 2}' '{}' '{}']
 ['{}' '{}' '{0, 1, 2}' '{}']
 ['{}' '{}' '{}' '{0, 1, 2}']]

## Random Abstract Matrix

In [16]:
AbstractMatrix.random((3, 3), ps3)

PSRing3 Matrix:
[['{2}' '{1}' '{0, 1, 2}']
 ['{0, 1, 2}' '{1}' '{1, 2}']
 ['{0, 1, 2}' '{}' '{1, 2}']]

## Get/Set Matrix Element Values

In [17]:
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

In [18]:
mat2[0, 1]

'{0, 2}'

In [19]:
mat2[2, 2] = '{0, 1, 2}'
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{0, 1, 2}']]

In [20]:
mat2[2, 2] = '{}'
mat2

PSRing3 Matrix:
[['{0, 1}' '{0, 2}' '{1, 2}']
 ['{0}' '{1, 2}' '{2}']
 ['{0, 1}' '{}' '{}']]

## Matrix Minor

In [21]:
mnr2 = mat2.minor(0,0)
mnr2

PSRing3 Matrix:
[['{1, 2}' '{2}']
 ['{}' '{}']]

## Abstract Cofactor Matrix

In [22]:
cof2 = mat2.cofactor_matrix()
cof2

PSRing3 Matrix:
[['{}' '{}' '{1}']
 ['{}' '{1}' '{0}']
 ['{1}' '{}' '{0, 1}']]

## Abstract Matrix Transpose

In [23]:
cof2_trans = cof2.transpose()
cof2_trans

PSRing3 Matrix:
[['{}' '{}' '{1}']
 ['{}' '{1}' '{}']
 ['{1}' '{0}' '{0, 1}']]

## Abstract Matrix Determinant

In [24]:
mat2.determinant()

'{1}'

Here's a breaksdown of the basic computations required to get the determinant.

NOTE: Addition & multiplication for the ring, ps3, is commutative, so the order of adds and mults is irrelevant, below.

In [25]:
minor_det_0 = ps3.sub(ps3.mult('{1, 2}', '{}'), ps3.mult('{2}', '{}'))
minor_det_0

'{}'

In [26]:
minor_det_1 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{2}', '{0, 1}'))
minor_det_1

'{}'

In [27]:
minor_det_2 = ps3.sub(ps3.mult('{0}', '{}'), ps3.mult('{1, 2}', '{0, 1}'))
minor_det_2

'{1}'

In [28]:
det = ps3.sub(ps3.add(ps3.mult('{0, 1}', minor_det_0),
                      ps3.mult('{1, 2}', minor_det_2)),
              ps3.mult('{0, 2}', minor_det_1))
det

'{1}'

## Abstract Matrix Inverse

In [29]:
mat2_inv = mat2.inverse()
mat2_inv

PSRing3 Matrix:
[['{}' '{}' '{1}']
 ['{}' '{1}' '{}']
 ['{1}' '{}' '{1}']]

In [30]:
mat2 * mat2_inv

PSRing3 Matrix:
[['{1}' '{}' '{}']
 ['{}' '{1}' '{}']
 ['{}' '{}' '{1}']]

In [31]:
mat2_inv * mat2

PSRing3 Matrix:
[['{1}' '{}' '{}']
 ['{}' '{1}' '{}']
 ['{}' '{}' '{1}']]

In [32]:
ps3.one

'{0, 1, 2}'

In [33]:
foo = alg.generate_algebra_mod_n(4)

In [34]:
foo.one

'a1'

In [35]:
ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  17 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

In [36]:
AbstractMatrix.identity(4, ps3)

PSRing3 Matrix:
[['{0, 1, 2}' '{}' '{}' '{}']
 ['{}' '{0, 1, 2}' '{}' '{}']
 ['{}' '{}' '{0, 1, 2}' '{}']
 ['{}' '{}' '{}' '{0, 1, 2}']]